In [4]:
from openpyxl.workbook.workbook import Workbook 
import requests 
import json 
from datetime import datetime 

# 헤더값은 각자의 f12의 내용 참고해서 작성해야됨. 아래 코드의 값과 다를 수 있음. 
header = { 
    'accept': '*/*', 
    'accept-encoding': 'gzip, deflate, br', 
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7', 
    'cookie': 'mid=Yl4A5gALAAGkM1xRDCnHj_QMA0LC; ig_did=24E6B9F6-818E-450A-AD22-ACBE78D4F374; ig_nrcb=1; csrftoken=CUQ91TR4rkLcWI5GX7yItQDEMfCZLyX5; ds_user_id=52627331756; sessionid=52627331756%3A26RfWACrO824k6%3A18; rur="EAG\05452627331756\0541681864196:01f7e0a60374ee15f81170706387198314d0166585cc588474aa3ae552d60d7ce26df77b"', #교체
    'referer': "https://www.instagram.com/", 
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="100", "Google Chrome";v="100"', 
    'sec-ch-ua-mobile': '?0', 
    'sec-ch-ua-platform':'windows', 
    'sec-fetch-dest': 'empty', 
    'sec-fetch-mode': 'cors', 
    'sec-fetch-site': 'same-origin', 
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36', #교체
    'x-asbd-id': '198387', 
    'x-ig-app-id': '936619743392459', 
    'x-ig-www-claim': 'hmac.AR0xvMfbTxmrH54oW4AQdaa_T5SSYuD9uxm4nQtPyh-icvhY', #교체
    'x-requested-with': 'XMLHttpRequest' 
    } 


dataList = [] 
search = '섭지코지'
URL = 'https://www.instagram.com/explore/tags/'+search+'/?__a=1' 

# 개수확인용도 
pidCount = 0 

while(True): 

    res = requests.get(URL, headers =header) 
    res = res.json() 

    if 'next_page' not in res['data']['recent'].keys() or int(res['data']['recent']['next_page']) == 0: break 
    max_id = res['data']['recent']['next_max_id'] 


    for n in res['data']['recent']['sections']: 
        for m in ((n['layout_content']['medias'])): 
            # m : 인스타 게시글 1개 
            m = m['media'] 
            data = {} 
            data['pagePk'] = m['code'] 
            data['URL'] = 'https://www.instagram.com/p/'+ data['pagePk']+"/" 
            data['user_name'] = (m['user']['username']) 

            # 이미지/동영상 체크 
            keys = m.keys() 
            if 'carousel_media' in keys: 
                data['image'] = (m['carousel_media']) 
            else: 
                data['image'] = [{'image_versions2':{"candidates":[{'url'}]}}] 
                # data['image'] = [{'image_versions2':{"candidates":[{'url':'동영상업로드'}]}}] 
            
            dataList.append(data) 
            pidCount = pidCount+1 

            comments = m['comments']
            
            # print(m.keys())


            for i in range(len(comments)):
                if (len(comments) != 0):
                    print(str(pidCount) + '번 ' + str(i+1) + '  ' +'@' + data['user_name'] + ', 댓글 : ' + comments[i].get('text') + ', URL : ' + data['URL'])


    URL = 'https://www.instagram.com/explore/tags/'+search+'/?__a=1&max_id='+max_id 
    


2번 1  @kangeunsim_, 댓글 : 안녕하세요 선팔하고갑니다 제 피드에도 놀러와주실거죠^^?, URL : https://www.instagram.com/p/CchRC9vP-A2/
4번 1  @jejudojmt, 댓글 : 👍, URL : https://www.instagram.com/p/CchQWu9pxVQ/
6번 1  @97_gayun, 댓글 : #dailylook #ootd #선팔 #맞팔 #선팔맞팔 #좋아요 #좋아요반사 #좋아요테러 #좋반 #좋튀 #일상 #좋반테러 #소통 #소통해요 #데일리 #follow4like #follow4followback #followforfollowback #f4f #like4likes #likeforlike #fff, URL : https://www.instagram.com/p/CchOvd4BHAD/
6번 2  @97_gayun, 댓글 : 여기 휴일로 너무 좋지 않나요ㅎㅎ!!, URL : https://www.instagram.com/p/CchOvd4BHAD/
7번 1  @she.shed_, 댓글 : 🙌🏻, URL : https://www.instagram.com/p/CchOz0RLsZT/
7번 2  @she.shed_, 댓글 : @she.shed_ #제주도여행 #제주데이트 #제주도술집 #제주도카페 #제주가구공방 #제주원데이클래스 #원데이클래스 #원목소품 #소품샵 #신혼부부 #플레이스캠프제주 #제주게스트하우스 #제주쉐어하우스 #제주독채 #성산맛집 #성산와인 #성산위스키 #위스키 #위스키바 #몰트바 #성산일출봉 #섭지코지 #광치기해변, URL : https://www.instagram.com/p/CchOz0RLsZT/
9번 1  @bumtage, 댓글 : #범티지, URL : https://www.instagram.com/p/CchJohBL7B2/
9번 2  @bumtage, 댓글 : @bumtage #제주도빈티지샵 #서귀포빈티지샵
#제주여행 #제주가볼만한곳
#서귀포가볼만한곳 #서귀포맛집
#제주핫플 #제주한달살기 #제주살이

KeyError: 'comments'

In [ ]:
# 엑셀 다운로드 및 이미지 저장 

from openpyxl import Workbook 
wb = Workbook() 
ws = wb.active 


#시트생성 
sheet1 = wb['Sheet'] 
sheet1.title = '수집 데이터' 
sheet1['A1'] = '아이디' 
sheet1['B1'] = 'URL' 
sheet1['C1'] = '이미지' 

sheetIndex = 2 

for data in dataList: 
    sheet1['A'+str(sheetIndex)] = data['user_name'] 
    sheet1['B'+str(sheetIndex)] = data['URL'] 

    # 이미지링크 엑셀에 저장 
    for imgUrl in data['image']: 
        url = imgUrl['image_versions2']['candidates']['url'] 
        # url = imgUrl['image_versions2']['candidates'][0]['url'] 
        sheet1['C'+str(sheetIndex)] = url 

        # 이미지 폴더에 저장 
        if not os.path.exists("./images/"+data['user_name']): 
            os.mkdir("./images/"+data['user_name']) 
            
        if url not in '동영상업로드': 
            r = requests.get(url) 
            file = open("./images/"+data['user_name']+'/'+data['user_name']+str(sheetIndex)+".jpg","wb") 
            file.write(r.content) 
            file.close() 
            
        sheetIndex = sheetIndex+1 

wb.save('./test1.xlsx')


TypeError: list indices must be integers or slices, not str